In [17]:
import pandas as pd
import numpy as np
import datetime as dt
import dateutil as du
import statsmodels.formula.api as sm
import sklearn as sk
import sklearn.ensemble as ske
from sklearn.decomposition import PCA
import scipy as sp
import pickle as pc
import matplotlib.pylab as py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [18]:
#Best fit function
def linear_func(x, A, b):
    return A * x + b

In [19]:
# Show plots of interesting correlations
def plot_patients(patients):
    columns = ['HR','HCT','FiO2','Glucose','Temp','WBC','GCS','Urine','Age','Gender','Length_of_stay'] #Select the columns of interest
    for i in np.arange(len(columns)):
        for j in np.arange(i+1,len(columns)):
            if(i==j):
                break
            else:       
                fig = plt.figure(figsize=(10,10))
                params, params_covariance = sp.optimize.curve_fit(linear_func, patients[columns[i]].astype('float').values, patients[columns[j]].astype('float').values,p0=[2, 2])
                targets = [ 'Alive','Dead']
                colors = ['g', 'r']
                for target, color in zip(targets,colors):
                    indicesToKeep = patients.index[patients.DeathStatus==target]
                    plt.scatter(patients.loc[indicesToKeep, columns[i]]
                               , patients.loc[ indicesToKeep, columns[j]]
                               , c = color
                               , s = 50)
                plt.legend(targets)
                plt.grid()
                plt.plot(patients[columns[i]],linear_func(patients[columns[i]].astype('float').values,params[0],params[1]),color='red')
                plt.xlabel(columns[i],size=30)
                plt.ylabel(columns[j],size=30)
                corr, p_value = sp.stats.pearsonr(patients[columns[i]],patients[columns[j]])
                plt.title('Pearson R correlation Coeff: ' + str(round(corr,3)) + ';    p-value: ' + str(round(p_value,3)) + ';    Statistically Signficant: ' + str('Yes' if p_value<0.05 else 'No'), size=20 )

    return True

In [20]:
#Removes outliers using Winsorize technique in 'HCT','FiO2','Glucose','Temp', and 'WBC', then plots
def remove_Outliers(patients):
    patients.HCT = pd.Series(sp.stats.mstats.winsorize(patients.HCT,limits=[0.02,0.02],inplace=False))
    patients.Glucose = pd.Series(sp.stats.mstats.winsorize(patients.Glucose,limits=[0.02,0.02],inplace=False))
    patients.FiO2 = pd.Series(sp.stats.mstats.winsorize(patients.FiO2,limits=[0.02,0.02],inplace=False))
    patients.Temp = pd.Series(sp.stats.mstats.winsorize(patients.WBC,limits=[0.02,0.02],inplace=False))
    patients.WBC = pd.Series(sp.stats.mstats.winsorize(patients.HCT,limits=[0.02,0.02],inplace=False))
    return patients

In [8]:
#Get paitents with selected columns
num_patients = 500 # Number of patients
columns = ['HR','HCT','FiO2','Glucose','Temp','WBC','GCS','Urine','Age','Gender','Length_of_stay','DeathStatus'] 
#Select the columns of interest
patients = pd.DataFrame(index=np.arange(num_patients),columns=columns)
outcome = pd.read_csv("Outcomes-a.txt",sep=',')
i=0
for j in np.arange(len(outcome)):
    if(i<num_patients): 
        patient = pd.read_csv(str(outcome.RecordID[j]) + ".txt",sep=',')
        skip=False
        for k in np.arange(len(columns)):
            
#     Skip patient if not all the selected columns have values
         if(k < 9) and (not columns[k] in np.unique(patient.Parameter).tolist()):
        
#     print('Missing: ' + columns[k])
             skip=True
      if(not skip):
            
#  Populate the selected columns for this patient
    patients.iloc[i].HR = patient[patient.Parameter =='HR'].iloc[-1].Value
    patients.iloc[i].HCT = patient[patient.Parameter =='HCT'].iloc[-1].Value
    patients.iloc[i].Glucose = patient[patient.Parameter =='Glucose'].iloc[-1].Value
    patients.iloc[i].FiO2 = patient[patient.Parameter =='FiO2'].iloc[-1].Value
    patients.iloc[i].Temp = patient[patient.Parameter =='Temp'].iloc[-1].Value
    patients.iloc[i].WBC = patient[patient.Parameter =='WBC'].iloc[-1].Value
    patients.iloc[i].GCS = patient[patient.Parameter =='GCS'].iloc[-1].Value
    patients.iloc[i].Urine = patient[patient.Parameter =='Urine'].iloc[-1].Value
    patients.iloc[i].Age = patient[patient.Parameter =='Age'].iloc[0].Value
    patients.iloc[i].Gender = patient[patient.Parameter =='Gender'].iloc[0].Value
    patients.iloc[i].Length_of_stay = outcome[outcome.RecordID ==outcome.RecordID[j]].Length_of_stay.values[0]
    patients.iloc[i].DeathStatus = outcome[outcome.RecordID ==outcome.RecordID[j]]['In-hospital_death'].values[0]
    i = i+1


FileNotFoundError: File b'Outcomes-a.txt' does not exist

In [11]:
#Add patient number
patients.insert(loc=0,column='patient_num',value=pd.Series(np.arange(num_patients)))
# Now we have num_paitents unique patients with 'HR','HCT','FiO2','Glucose','Temp','WBC','GCS','Age','Gender','Length_of_stay', and 'DeathStatus'

#Change the flag to Alive and Dead
patients.loc[patients.DeathStatus==0,'DeathStatus'] ='Alive'
patients.loc[patients.DeathStatus==1,'DeathStatus'] ='Dead'

#Raw data plots
plot_patients(patients)

#Winsorized data plots
plot_patients(remove_Outliers(patients))


ValueError: cannot insert patient_num, already exists

In [12]:
#PCA
features = ['HR','HCT','FiO2','Glucose','Temp','WBC','GCS','Urine','Age','Gender','Length_of_stay']
# Separating out the features
x = remove_Outliers(patients).loc[:, features].values
# Separating out the target
y = patients.loc[:,['DeathStatus']].values
# Standardizing the features
x = sk.preprocessing.StandardScaler().fit_transform(x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [13]:
#Show side by side after standardizing
remove_Outliers(patients).loc[:5, features]
pd.DataFrame(x,columns=features).loc[:5, features]


,HR,HCT,FiO2,Glucose,Temp,WBC,GCS,Urine,Age,Gender,Length_of_stay
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:

#Run PCA
pca = sk.decomposition.PCA()
principalComponents = pca.fit_transform(x)
components= ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', 'principal component 6', 'principal component 7', 'principal component 8', 'principal component 9', 'principal component 10', 'principal component 11']
principalDf = pd.DataFrame(data = principalComponents, columns = components)

#Concatenate output or response column to the prinicipal components dataframe
finalDf = pd.concat([principalDf, patients[['DeathStatus']]], axis = 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [15]:
#Plot Components against each other and review the data points for alive and dead patients
for i in np.arange(len(components)):
    for j in np.arange(i+1,len(components)):
        if(i==j):
            break
        else:       
            fig = plt.figure(figsize = (10,10))
            ax = fig.add_subplot(1,1,1)
            ax.set_xlabel(components[i], fontsize = 15)
            ax.set_ylabel(components[j], fontsize = 15)
            ax.set_title('2 component PCA', fontsize = 20)
            targets = [ 'Alive','Dead']
            colors = ['g', 'r']
            for target, color in zip(targets,colors):
                indicesToKeep = finalDf.index[finalDf.DeathStatus==target]
                ax.scatter(finalDf.loc[indicesToKeep, components[i]]
                           , finalDf.loc[ indicesToKeep, components[j]]
                           , c = color
                           , s = 50)
            ax.legend(targets)
            ax.grid()

NameError: name 'components' is not defined

In [16]:
#Get the correlations between the components
compsVariances = pd.DataFrame(pca.get_covariance(),index=components,columns=components)
compsVariances.to_excel('covariance_matrix.xlsx',index=True)

# Get Eignevalues in the descending order and plot cumutively
pca.explained_variance_ratio_
fig = plt.figure(figsize=(10,10))
plt.plot(np.arange(len(components)),np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3)*100))
plt.ylabel('% Variance Explained')
plt.xlabel('# of Components')
plt.title('PCA Analysis')
plt.ylim(30,100.5)
plt.grid()
plt.style.context('seaborn-whitegrid')

AttributeError: 'PCA' object has no attribute 'components_'